In [1]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import math
import time

from transformers import Trainer, is_torch_xla_available
from transformers.trainer_utils import PredictionOutput, speed_metrics

if is_torch_xla_available():
    import torch_xla.core.xla_model as xm
    import torch_xla.debug.metrics as met


class QuestionAnsweringTrainer(Trainer):
    def __init__(self, *args, eval_examples=None, post_process_function=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.eval_examples = eval_examples
        self.post_process_function = post_process_function

    def evaluate(self, eval_dataset=None, eval_examples=None, ignore_keys=None, metric_key_prefix: str = "eval"):
        eval_dataset = self.eval_dataset if eval_dataset is None else eval_dataset
        eval_dataloader = self.get_eval_dataloader(eval_dataset)
        eval_examples = self.eval_examples if eval_examples is None else eval_examples

        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
        start_time = time.time()
        try:
            output = eval_loop(
                eval_dataloader,
                description="Evaluation",
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
                metric_key_prefix=metric_key_prefix,
            )
        finally:
            self.compute_metrics = compute_metrics
        total_batch_size = self.args.eval_batch_size * self.args.world_size
        if f"{metric_key_prefix}_jit_compilation_time" in output.metrics:
            start_time += output.metrics[f"{metric_key_prefix}_jit_compilation_time"]
        output.metrics.update(
            speed_metrics(
                metric_key_prefix,
                start_time,
                num_samples=output.num_samples,
                num_steps=math.ceil(output.num_samples / total_batch_size),
            )
        )
        if self.post_process_function is not None and self.compute_metrics is not None and self.args.should_save:
            eval_preds = self.post_process_function(eval_examples, eval_dataset, output.predictions)
            metrics = self.compute_metrics(eval_preds)
            for key in list(metrics.keys()):
                if not key.startswith(f"{metric_key_prefix}_"):
                    metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)
            metrics.update(output.metrics)
        else:
            metrics = output.metrics
        if self.args.should_log:
            self.log(metrics)
        if self.args.tpu_metrics_debug or self.args.debug:
            xm.master_print(met.metrics_report())
        self.control = self.callback_handler.on_evaluate(self.args, self.state, self.control, metrics)
        return metrics

    def predict(self, predict_dataset, predict_examples, ignore_keys=None, metric_key_prefix: str = "test"):
        predict_dataloader = self.get_test_dataloader(predict_dataset)
        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
        start_time = time.time()
        try:
            output = eval_loop(
                predict_dataloader,
                description="Prediction",
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
                metric_key_prefix=metric_key_prefix,
            )
        finally:
            self.compute_metrics = compute_metrics
        total_batch_size = self.args.eval_batch_size * self.args.world_size
        if f"{metric_key_prefix}_jit_compilation_time" in output.metrics:
            start_time += output.metrics[f"{metric_key_prefix}_jit_compilation_time"]
        output.metrics.update(
            speed_metrics(
                metric_key_prefix,
                start_time,
                num_samples=output.num_samples,
                num_steps=math.ceil(output.num_samples / total_batch_size),
            )
        )
        if self.post_process_function is None or self.compute_metrics is None:
            return output
        predictions = self.post_process_function(predict_examples, predict_dataset, output.predictions, "predict")
        metrics = self.compute_metrics(predictions)
        for key in list(metrics.keys()):
            if not key.startswith(f"{metric_key_prefix}_"):
                metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)
        metrics.update(output.metrics)
        return PredictionOutput(predictions=predictions.predictions, label_ids=predictions.label_ids, metrics=metrics)


In [3]:
import json

def convert_squad_format_to_flat(input_file, output_file):
    # Open and load the JSON data
    with open(input_file, "r", encoding="utf-8") as infile:
        data = json.load(infile)
    
    # Initialize a list for the flat format
    flat_data = []
    
    # Extract and process the structure
    for item in data["data"]:
        # Extract fields directly from the current structure
        flat_data.append({
            "id": item["id"],
            "context": item["context"],
            "question": item["question"],
            "answers": {
                "text": item["answers"]["text"],
                "answer_start": item["answers"]["answer_start"]
            },
            "title": item.get("title", "default-title")  # Default to "default-title" if 'title' is missing
        })
    
    # Save the flat JSON data
    with open(output_file, "w", encoding="utf-8") as outfile:
        json.dump(flat_data, outfile, indent=4, ensure_ascii=False)

# Example usage
convert_squad_format_to_flat("/kaggle/input/viquad/train.json", "processed_train.json")
convert_squad_format_to_flat("/kaggle/input/viquad/dev.json", "processed_dev.json")
convert_squad_format_to_flat("/kaggle/input/viquad/Private_Test_ref.json", "processed_test.json")


In [4]:
import os
import numpy as np
import json
import math
import time
from datasets import load_dataset
from transformers import (
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    TrainingArguments,
    TrainerCallback,
)
from transformers.trainer_utils import PredictionOutput, speed_metrics
import evaluate

# Define paths to datasets
train_file = "processed_train.json"
dev_file = "processed_dev.json"
test_file = "processed_test.json"

# Define models to fine-tune
models = {
    "mBERT": "bert-base-multilingual-cased",
    "XLM-R": "xlm-roberta-base",
}

# Load evaluation metric
metric = evaluate.load("squad_v2")


# Preprocess function for tokenizing input data
def prepare_train_features(examples, tokenizer, max_length=384, doc_stride=128):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]

        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


# Post-processing function
def postprocess_predictions(examples, features, predictions, mode="eval"):
    start_logits, end_logits = predictions

    results = []
    for example, feature in zip(examples, features):
        start_idx = np.argmax(start_logits[0])
        end_idx = np.argmax(end_logits[0])
        results.append(
            {
                "id": example["id"],
                "prediction_text": " ".join([str(x) for x in feature["input_ids"][start_idx:end_idx]]),
                "no_answer_probability": 0.0,  # Default value
            }
        )

    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]

    return {"predictions": results, "references": references}



In [5]:
import wandb
wandb.login(key="537b58289152906ce9ec9f8986e2d8774d988784")


# Load datasets
datasets = load_dataset("json", data_files={"train": train_file, "validation": dev_file, "test": test_file})

# Loop through models and train each
for model_name, model_checkpoint in models.items():
    print(f"Starting fine-tuning for {model_name}...")

    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

    # Preprocess datasets
    tokenized_datasets = datasets.map(
        lambda x: prepare_train_features(x, tokenizer),
        batched=True,
        remove_columns=datasets["train"].column_names,
    )

    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"./results/{model_name}",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=3e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir=f"./logs/{model_name}",
        logging_steps=100,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
    )

    # Use custom QuestionAnsweringTrainer
    trainer = QuestionAnsweringTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        eval_examples=datasets["validation"],
        post_process_function=postprocess_predictions,
        #compute_metrics=lambda p: metric.compute(predictions=p.predictions, references=p.label_ids),
        compute_metrics=lambda p: metric.compute(
            predictions=[pred["prediction_text"] for pred in p["predictions"]],
            references=[ref["answers"] for ref in p["references"]],
        )
    )

    # Train and evaluate
    trainer.train()
    eval_metrics = trainer.evaluate()
    print(f"Validation Results for {model_name}: {eval_metrics}")

    # Predict on test set
    test_results = trainer.predict(tokenized_datasets["test"], datasets["test"])
    print(f"Test Results for {model_name}: {test_results.metrics}")

    # Save predictions
    output_path = f"./results/{model_name}_test_predictions.json"
    with open(output_path, "w") as f:
        json.dump(test_results.predictions, f, indent=4)

print("Training and evaluation completed for all models.")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 22521117 (duongtienpham). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Starting fine-tuning for mBERT...


...

Map:   0%|          | 0/5692 [00:00<?, ? examples/s]

Map:   0%|          | 0/7301 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss


AttributeError: 'list' object has no attribute 'predictions'